---------------------

Methode fichier par fichier 

- encore sous forme de créa de DataFrame collectif (plusieurs fichiers assemblés)
pour vérifier le traitement des données. 

-----------------------

* [Exploration et mise au propre des donnees](#explo)
* [Fonction xml to bdd](#bdd)


In [7]:
import os 
import gzip 
import glob 
import xmltodict 
import polars as pl 
import pandas as pd 
import pyarrow as pa 

pd.set_option('display.max_columns',100)

colonnes_pd_budget = ['Id_Fichier',
    'Nomenclature',
    'Exer' ,
    'TypOpBudg' , #des 2 et des 1
    'Operation',
    'Nature',
    'ContNat',
    'LibCpte',
    'Fonction',
    'ContFon',
    'ArtSpe', 
    'CodRD', 
    'MtBudgPrec', 
    'MtRARPrec', 
    'MtPropNouv', 
    'MtPrev', 
    'OpBudg',
    'CredOuv', 
    'MtReal', 
    'MtRAR3112', 
    'ContOp',
    'OpeCpteTiers',
    'MtSup',
    'APVote',
    'Brut',
    'BudgetHorsRAR',
    'Comp',
    'ICNE',
    'ICNEPrec',
    'MtOpeCumul',
    'MtOpeInfo',
    'Net',
    'ProdChaRat',
    'RARPrec',
    'CaracSup',
    'TypOpe',
    'Section',
    'ChapSpe',
    'ProgAutoLib',
    'ProgAutoNum',
    'VirCredNum',
    'CodeRegion']

colonnes_doc_budgetaire = [
 'Id_Fichier',
 'Nomenclature',
 'Exer',
 'IdColl',
 'Siren',
 'CodColl',
 'LibelleColl',
 'DteStr',
 '@date',
 'DteDec',
 'DteDecEx',
 'NumDec',
 'IdPost',
 'LibellePoste',
 'LibelleEtabPal',
 'IdEtabPal',
 'LibelleEtab',
 'IdEtab',
 'NatDec',
 'NatVote', 	
 'OpeEquip', 
 'CodInseeColl',	
 'VoteFormelChap', 	
 'TypProv', 	
 'BudgPrec',
 'RefProv',	
 'ReprRes', 	
 'NatFonc', 	
 'PresentationSimplifiee', 	
 'DepFoncN2',	
 'RecFoncN2' ,	
 'DepInvN2' ,	
 'RecInvN2' ,	
 'CodTypBud',
 'CodBud',
 'ProjetBudget',  	
 'Affect',
 'SpecifBudget',
 'FinJur',	 	
 '@md5',	
 '@sha1']



In [20]:
def _isolement_id(fichier) : 
 '''Extrait l'id du fichier
 Il y a un if dans le cas où le fichier est sous le format 
 " 20201101-23430 " 

 Renvoie un str. 
 '''

 val_id_fichier_source = fichier.split("/")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def extraction_annexe(chemin_annexe, dict_metadonnees) : 
 liste_annexe = []
 for row in chemin_annexe : 
  liste_par_ligne = {}
  for a, b in row.items() : 
   liste_par_ligne.update({a : b.get('@V')})
   liste_par_ligne.update(dict_metadonnees)
  liste_annexe.append(liste_par_ligne)
 return liste_annexe 

def extraction_donnees(chemin) : 
 dict_annexe = {}
 for a, b in chemin.items() : 
   dict_annexe.update({a : b.get('@V')})
 return dict_annexe 

def extraction_lignes_budget_liste(chemin, dict_id) :
 liste_budget = []
 for lignes in chemin : 
  dict_ligne = {}
  dict_ligne.update(dict_id)
  for a, b in lignes.items() :
     if a not in ['MtSup', 'CaracSup'] : 
       dict_ligne.update({a : b.get('@V')}) 
 
     elif a == 'MtSup' : 
       dict_ligne.update({a : b})
       type_m = lignes.get('MtSup')
 
       if isinstance(type_m, dict) : 
        dict_ligne.update({type_m.get('@Code') : type_m.get('@V')})
 
       elif isinstance(type_m, list) : 
          for j in b : 
           dict_ligne.update({j.get('@Code') : j.get('@V')})
 
     elif a == 'CaracSup' :   
       dict_ligne.update({a : b})
       type_c = lignes.get('CaracSup')
 
       if isinstance(type_c, dict) :
        dict_ligne.update({type_c.get('@Code') : type_c.get('@V')})
 
       elif isinstance(type_c, list) : 
          for j in b : 
           dict_ligne.update({j.get('@Code') : j.get('@V')})

  liste_budget.append(dict_ligne)
 return liste_budget

def extraction_budget(fichier_parse, dict_id) : 
 ''' Extrait toutes les données budgetaires, y compris carac et mtsup '''
 lignes_budget = fichier_parse['DocumentBudgetaire']['Budget']['LigneBudget'] 

 if isinstance(lignes_budget, dict) : 
  donnees_budget_prep = extraction_donnees(lignes_budget)
  donnees_budget_prep.update(dict_id)
  donnees_budget = [donnees_budget_prep]

 elif isinstance(lignes_budget, list) : 
  donnees_budget = extraction_lignes_budget_liste(lignes_budget, dict_id)

 df_budget = pd.DataFrame(donnees_budget)
 df_colonnes = pd.DataFrame(columns=colonnes_pd_budget)
 df_budget_sans_schema = pd.concat([df_colonnes, df_budget])
 df_budget_propre = nettoyage_budget(df_budget_sans_schema)
 return df_budget_propre 



In [21]:
def extraction_document_budgetaire(fichier_parse, dictionnaire_id) : 
  ''' Extrait les métadonnées du fichier pour la table document_budgetaire '''
  blocbudget = extraction_donnees(fichier_parse['DocumentBudgetaire']['Budget']['BlocBudget'])
  entetedocbudg = extraction_donnees(fichier_parse['DocumentBudgetaire']['EnTeteDocBudgetaire'])
  entetebudget = extraction_donnees(fichier_parse['DocumentBudgetaire']['Budget']['EnTeteBudget'])
  scellement = fichier_parse['DocumentBudgetaire']['Scellement']

  liste_fichier = [{**blocbudget, **entetedocbudg, **entetebudget, **scellement, **dictionnaire_id}]
  df_doc_budgetaire_prep_1 = pd.DataFrame(liste_fichier)
  df_colonnes_budg = pd.DataFrame(columns= colonnes_doc_budgetaire)
  df_doc_budgetaire_prep_2 = pd.concat([df_colonnes_budg, df_doc_budgetaire_prep_1])
  df_doc_budgetaire = nettoyage_doc_budg(df_doc_budgetaire_prep_2)
  return df_doc_budgetaire

## Exploration et nettoyage des DataFrames <a class="anchor" id="explo"></a>

In [3]:
def extraction_annexe_concours(fichier_parse, dictionnaire_id) : 
 ''' Extrait les données de l'annexe concours pour la table correspondante 
 !! Necessite un try si Data_concours n'est pas dans le fichier '''
 concours = fichier_parse['DocumentBudgetaire']['Budget']['Annexes']['DATA_CONCOURS']['CONCOURS']
 liste_donnees_concours = extraction_annexe(concours, dictionnaire_id)
 df_concours = pd.DataFrame(liste_donnees_concours)
 return df_concours


In [125]:
def nettoyage_budget(df) : 
 df['Id_Fichier'] = df['Id_Fichier'].astype('Int32')
 df['Nomenclature'] = df['Nomenclature'].astype(str)
 df['Exer'] = df['Exer'].astype('Int16')
 df['TypOpBudg'] = df['TypOpBudg'].astype('Int32')
 #df['Operation'] = df['Operation'].astype('Int16') #12VEM488 , aire de jeux etc
 df['ArtSpe'] = df['ArtSpe'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['MtBudgPrec'] = df['MtBudgPrec'].astype(float)
 df['MtRARPrec'] = df['MtRARPrec'].astype(float)
 df['MtPropNouv'] = df['MtPropNouv'].astype(float)
 df['MtPrev'] = df['MtPrev'].astype(float)
 df['OpBudg'] = df['OpBudg'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['CredOuv'] = df['CredOuv'].astype(float)
 df['MtReal'] = df['MtReal'].astype(float)
 df['MtRAR3112'] = df['MtRAR3112'].astype(float)
 #df['ContOp'] = df['ContOp'].replace('', None).astype('Int16') #contient des '16 01'
 df['APVote'] = df['APVote'].astype(float)
 df['Brut'] = df['Brut'].astype(float)
 df['BudgetHorsRAR'] = df['BudgetHorsRAR'].astype(float)
 df['ICNE'] = df['ICNE'].astype(float)
 df['ICNEPrec'] = df['ICNEPrec'].astype(float)
 df['MtOpeCumul'] = df['MtOpeCumul'].astype(float)
 df['MtOpeInfo'] = df['MtOpeInfo'].astype(float)
 df['Net'] = df['Net'].astype(float)
 df['ProdChaRat'] = df['ProdChaRat'].astype(float)
 df['TypOpe'] = df['TypOpe'].astype('Int32')
 df['CodeRegion'] = df['CodeRegion'].astype('Int16')
 return df 

def nettoyage_doc_budg(df) : 
 df = df.drop(columns=['NatCEPL', 'Departement'])
 df = df.rename(columns={'IdColl' : 'Siret', '@date' : 'date_precise',
                    '@md5' : 'md5', '@sha1' : 'sha1'}) 
 
 df['Id_Fichier'] = df['Id_Fichier'].astype('Int32')
 df['Exer'] = df['Exer'].astype('Int16')
 df['Siren'] = df['Siret'].str.slice(0,9)
 df['Siret'] = df['Siret'].astype('Int64')
 df['Siren'] = df['Siren'].astype('Int64')
 df['DteStr'] = pd.to_datetime(df['DteStr'])
 df['DteDec'] = pd.to_datetime(df['DteDec'], errors= 'coerce')
 df['date_precise'] = pd.to_datetime(df['date_precise'], format='ISO8601', utc= True)
 df['DteDecEx'] = pd.to_datetime(df['DteDecEx'])
 df['IdEtabPal'] = df['IdEtabPal'].astype('Int64')
 df['IdEtab'] = df['IdEtab'].astype('Int64')
 df['OpeEquip'] = df['OpeEquip'].replace(
     {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['VoteFormelChap'] = df['VoteFormelChap'].replace(
     {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['TypProv'] = df['TypProv'].astype('Int16')
 df['BudgPrec'] = df['BudgPrec'].astype('Int16')
 df['ReprRes'] = df['ReprRes'].astype('Int16')
 df['NatFonc'] = df['NatFonc'].astype('Int16')
 df['PresentationSimplifiee'] = df['PresentationSimplifiee'].replace(
     {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['DepFoncN2'] = df['DepFoncN2'].astype(float)
 df['RecFoncN2'] = df['RecFoncN2'].astype(float)
 df['DepInvN2'] = df['DepInvN2'].astype(float)
 df['RecInvN2'] = df['RecInvN2'].astype(float)
 df['ProjetBudget'] = df['ProjetBudget'].replace(
     {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
 df['SpecifBudget'] = df['SpecifBudget'].astype('Int64')
 df['FinJur'] = df['FinJur'].astype('Int64')
 return df  

 

--------------------------------------------------------------

Optionnel, explo : 

In [123]:
def multi_document(chemin_des_xml) :
 ''' Changement à faire : 
 Faire extraire l'Id_Fichier'''
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_des_xml, "*.gz"))
 #connection à la table
 liste_df = []

 for fichier in chemin_xml_entree_glob : 
  id_fichier = _isolement_id(fichier)
  #print(id_fichier)
  #Necessite verif dans tables
  if id_fichier is None : 
   print('vide')
   pass 
  else : 
   try : 
    #print('etape 2')
    fichier_parse = parse_fichier(fichier)
    dict_id = {'Id_Fichier' : id_fichier}
    #print(dict_metadonnees)
    df_doc = extraction_document_budgetaire(fichier_parse, dict_id)
    liste_df.append(df_doc)
    #insertion dans table
   except Exception as e : 
     print(id_fichier, 'erreur')
     print(e)
 
 df_mega = pd.concat(liste_df)
 return df_mega 

In [103]:
df_doc1 = multi_document(chemin_20)

775775 erreur
no element found: line 1, column 0


In [62]:
df_doc1['FinJur'].value_counts()

FinJur
910811363         2
9308127722        1
05                1
21590017600011    1
380790386         1
Name: count, dtype: int64

In [113]:
#df_doc1 = df_doc1.drop(columns=['NatCEPL', 'Departement'])
df_doc1 = df_doc1.rename(columns={'IdColl' : 'Siret'})

In [ ]:
df_doc1['Id_Fichier'] = df_doc1['Id_Fichier'].astype('Int32')
df_doc1['Exer'] = df_doc1['Exer'].astype('Int16')
#df_doc1['Siren'] = df_doc1['Siret'].str.slice(0,9)
#df_doc1['Siret'] = df_doc1['Siret'].astype('Int64')
#df_doc1['Siren'] = df_doc1['Siren'].astype('Int64')
df_doc1['DteStr'] = pd.to_datetime(df_doc1['DteStr'])
df_doc1['DteDec'] = pd.to_datetime(df_doc1['DteDec'], errors= 'coerce')
df_doc1['@date'] = pd.to_datetime(df_doc1['@date'], format='ISO8601', utc= True)
df_doc1['DteDecEx'] = pd.to_datetime(df_doc1['DteDecEx'])
df_doc1['IdEtabPal'] = df_doc1['IdEtabPal'].astype('Int64')
df_doc1['IdEtab'] = df_doc1['IdEtab'].astype('Int64')
df_doc1['OpeEquip'] = df_doc1['OpeEquip'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
df_doc1['VoteFormelChap'] = df_doc1['VoteFormelChap'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
df_doc1['TypProv'] = df_doc1['TypProv'].astype('Int16')
df_doc1['BudgPrec'] = df_doc1['BudgPrec'].astype('Int16')
df_doc1['ReprRes'] = df_doc1['ReprRes'].astype('Int16')
df_doc1['NatFonc'] = df_doc1['NatFonc'].astype('Int16')
df_doc1['PresentationSimplifiee'] = df_doc1['PresentationSimplifiee'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
df_doc1['DepFoncN2'] = df_doc1['DepFoncN2'].astype(float)
df_doc1['RecFoncN2'] = df_doc1['RecFoncN2'].astype(float)
df_doc1['DepInvN2'] = df_doc1['DepInvN2'].astype(float)
df_doc1['RecInvN2'] = df_doc1['RecInvN2'].astype(float)
df_doc1['ProjetBudget'] = df_doc1['ProjetBudget'].replace(
    {'0' : False, '1' : True, 'false' : False, "true" : True}).astype(bool)
df_doc1['SpecifBudget'] = df_doc1['SpecifBudget'].astype('Int64')
df_doc1['FinJur'] = df_doc1['FinJur'].astype('Int64')


df_doc1.head()

In [124]:
df_doc1.dtypes

Id_Fichier                              Int32
Nomenclature                           object
Exer                                    Int16
Siret                                   Int64
Siren                                   Int64
CodColl                                object
LibelleColl                            object
DteStr                         datetime64[ns]
@date                     datetime64[ns, UTC]
DteDec                         datetime64[ns]
DteDecEx                       datetime64[ns]
NumDec                                 object
IdPost                                 object
LibellePoste                           object
LibelleEtabPal                         object
IdEtabPal                               Int64
LibelleEtab                            object
IdEtab                                  Int64
NatDec                                 object
NatVote                                object
OpeEquip                                 bool
CodInseeColl                      

In [ ]:
df8['@date'] = pd.to_datetime(df8['@date'], format='ISO8601', utc= True)
df8.head()

In [100]:
df_doc1['FinJur'].value_counts()

FinJur
910811363         2
9308127722        1
05                1
21590017600011    1
380790386         1
Name: count, dtype: int64

In [126]:
chemin_20 = '../../fichiers20/todo_xml_20/'

def multi_budget(chemin_des_xml) :
 ''' Changement à faire : 
 - Enlever la liste, ne plus faire de concat
 - Faire une fonction de select dans la table 
 - Faire une fonction d'insertion dans la table
 - Voir les 7 erreurs '''  
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_des_xml, "*.gz"))
 #connection à la table
 liste_df = []

 for fichier in chemin_xml_entree_glob : 
  id_fichier = _isolement_id(fichier)
  #print(id_fichier)
  #Necessite verif dans tables
  if id_fichier is None : 
   print('vide')
   pass 
  else : 
   try : 
    #print('etape 2')
    fichier_parse = parse_fichier(fichier)
    chemin_exer = fichier_parse['DocumentBudgetaire']['Budget']['BlocBudget']['Exer']
    chemin_nomenclature = fichier_parse['DocumentBudgetaire']['Budget']['EnTeteBudget']['Nomenclature']
    dict_metadonnees = {'Id_Fichier' : id_fichier, 
                        'Nomenclature' : chemin_nomenclature.get('@V'),
                        'Exer' : chemin_exer.get('@V')}
    #print(dict_metadonnees)
    df_budget = extraction_budget(fichier_parse, dict_metadonnees)
    liste_df.append(df_budget)
    #insertion dans table
   except Exception as e : 
     print(id_fichier, 'erreur')
     print(e)
 
 df_mega = pd.concat(liste_df)
 return df_mega 

In [6]:
der = multi_budget(chemin_20)

775775 erreur
no element found: line 1, column 0


In [ ]:
der.head()

In [128]:
der['Comp'].value_counts()

Comp
0    22
Name: count, dtype: int64

In [ ]:
der[(~der['RARPrec'].isna()) | (~der['RARprec'].isna()) ][['Id_Fichier','RARPrec', 'RARPrec']]

In [23]:
der['CodeRegion'].value_counts()

CodeRegion
1    1427
2     733
4     663
3     525
5     140
Name: count, dtype: int64

In [ ]:
der[~der['APVote'].isna()]

In [ ]:
f719853 = '../../fichiers20/todo_xml_20/760357.xml.gz'
parse_fichier(f719853)

# Passage bdd <a class="anchor" id="bdd"></a>

Objectif actuel : 

- Selectionner un fichier
- Extraire son ID
- L'ouvrir 
- Chercher les métadonnées
- S'il n'y a pas d'erreurs (fichier corrompu par exemple) chercher les données budgets

- Traiter les deux blocs, préparer les données
- Insérer ces données dans la bdd 

Objectifs plus tardif : 

- Les annexes (boucle par annexe dans le fichier ? Systeme de dictionnaires ? )
- Ajouter le transcodage dans le github 

In [1]:
import psycopg2
from psycopg2 import sql

con = psycopg2.connect(dbname='db_v1',
      user= 'postgres',
       host='',
       port = '5432')

In [5]:
def verification_fichier_dans_bdd(conn, id_fichier) : 
 ''' Renvoie le nombre de fois où l'ID Fichier est dans Doc_Budgetaire.
 Autrement dit, si 0, on envoie, si 1+ on annule.'''

 try : 
  conn = con
  cursor = conn.cursor()
  id_fichier_int = int(id_fichier)
  demande_verif = f'''SELECT COUNT(*) FROM doc_budget WHERE Id_Fichier = {id_fichier_int}'''
  cursor.execute(demande_verif)
  resultat = cursor.fetchone()[0]

 except psycopg2.Error as e : 
    print('erreur, rollback en cours')
    conn.rollback()

 finally : 
    cursor.close()

 return resultat

In [ ]:
def creation_dict_metadonnees_basiques(fichier_parse, id_fichier) : 
 ''' Crée un dictionnaire comportant les métadonnées du fichier nécessaire à la table
 Budget et aux annexes, ça permet de faciliter les join avec les libelle comptable'''
 chemin_exer = fichier_parse['DocumentBudgetaire']['Budget']['BlocBudget']['Exer']
 chemin_nomenclature = fichier_parse['DocumentBudgetaire']['Budget']['EnTeteBudget']['Nomenclature']
 dict_metadonnees = {'Id_Fichier' : id_fichier, 
                    'Nomenclature' : chemin_nomenclature.get('@V'),
                    'Exer' : chemin_exer.get('@V')}

return dict_metadonnees
 

In [ ]:
chemin_20 = '../../fichiers20/todo_xml_20/'

def xml_to_bdd(chemin_des_xml) :
 ''' Ajouts requis : les 30 annexes'''

 #Connect
 conn = psycopg2.connect(dbname='db_v1',
      user= 'postgres',
       host='',
       port = '5432')
 
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_des_xml, "*.gz"))

 #Travail individuel
 for fichier in chemin_xml_entree_glob : 
  id_fichier = _isolement_id(fichier)

  if id_fichier is None : 
   print('vide')
   pass 

  else : 
   presence_id_bdd = verification_fichier_dans_bdd(int(i))

   if presence_id_bdd > 0 :
    print(id_fichier, ', déjà présent dans bdd')
    pass 

   elif presence_id_bdd == 0 : 
    try : 
     fichier_parse = parse_fichier(fichier)
     dict_id = {'Id_Fichier' : id_fichier}
     dict_metadonnees = creation_dict_metadonnees_basiques(fichier_parse, id_fichier)

     df_doc_budget = extraction_document_budgetaire(fichier_parse, dict_id)
     df_budget = extraction_budget(fichier_parse, dict_metadonnees)

     #Insertion, ajouter un try ?
     df_doc_budget.to_sql('doc_budget', conn, if_exists = 'append', index = False)
     df_budget.to_sql('bloc_budget', conn, if_exists = 'append', index = False)
    except Exception as e : 
      print(id_fichier, 'erreur')
      print(e)
 
